<a href="https://colab.research.google.com/github/abDesgnr/FirstGitHub/blob/master/tf_tut_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# May 4, 2023
# Code example from TensorFlow tutorial:

#This tutorial shows how to load and preprocess an image dataset in three ways:

# 1) you will use high-level Keras preprocessing utilities (such as 
#    tf.keras.utils.image_dataset_from_directory) and layers (such 
#    as tf.keras.layers.Rescaling) to read a directory of images on disk.
# 2) you will write your own input pipeline from scratch using tf.data.
# 3) you will download a dataset from the large catalog available in 
#    TensorFlow Datasets.

import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib

print("TensorFlow v" + str(tf.__version__))

# Use Google's dataset of several thousand photos of flowers
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)
data_dir = pathlib.Path(archive).with_suffix('')

# After downloading (218MB), you should now have a copy of the 
# flower photos available. There are 3,670 total images:
image_count = len(list(data_dir.glob('*/*.jpg')))
print("Number of images " + str(image_count))

# Each directory contains images of that type of flower. Here are some roses:
roses = list(data_dir.glob('roses/*'))
#PIL.Image.open(str(roses[0]))

# Load these images off disk using Keras utility
# First, define some parameters for the loader:
batch_size = 32
img_height = 180
img_width = 180

# It's good practice to use a validation split when developing your model.
# You will use 80% of the images for training and 20% for validation.
# Training data
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Validation data
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# You can find the class names in the class_names attribute on these datasets.
class_names = train_ds.class_names
print(class_names)

# Standardize the data
# The RGB channel values are in the [0, 255] range. This is not ideal 
# for a neural network; in general you should seek to make your input 
# values small.
# standardize values to be in the [0, 1] range
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Configure the dataset for performance
# Use buffered prefetching so you can yield data from disk without 
# having I/O become blocking. Following are 2 important methods:
# 1) Dataset.cache keeps the images in memory after they're loaded off 
# disk during the first epoch
# 2) Dataset.prefetch overlaps data preprocessing and model execution 
# while training.
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Train Model
# The Sequential model consists of three convolution blocks 
# (tf.keras.layers.Conv2D) with a max pooling layer 
# (tf.keras.layers.MaxPooling2D) in each of them. There's a 
# fully-connected layer (tf.keras.layers.Dense) with 128 units on 
# top of it that is activated by a ReLU activation function ('relu'). 
# This model has not been tuned in any way—the goal is to show you 
# the mechanics using the datasets you just created.
# To learn more about image classification, visit the Image 
# classification tutorial.
num_classes = 5

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

# Choose the tf.keras.optimizers.Adam optimizer and 
# tf.keras.losses.SparseCategoricalCrossentropy loss function.
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

# Run fit() method
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)


TensorFlow v2.12.0
Number of images 3670
Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
Epoch 1/3
92/92 [==============================] - 174s 2s/step - loss: 1.3716 - accuracy: 0.4053 - val_loss: 1.1396 - val_accuracy: 0.5613
Epoch 2/3
92/92 [==============================] - 145s 2s/step - loss: 1.0838 - accuracy: 0.5708 - val_loss: 1.0521 - val_accuracy: 0.5749
Epoch 3/3
92/92 [==============================] - 145s 2s/step - loss: 0.9276 - accuracy: 0.6417 - val_loss: 0.9718 - val_accuracy: 0.5913
